## 	金融専門極性辞書を以下のサイトからダウンロード
* https://sites.google.com/socsim.org/izumi-lab/tools/%E9%87%91%E8%9E%8D%E5%B0%82%E9%96%80%E6%A5%B5%E6%80%A7%E8%BE%9E%E6%9B%B8
* ./data以下に配置

In [1]:
import glob  # 指定ディレクトリ以下のファイルを得る
import json  # jsonデータ用

import MeCab
import pandas as pd
from tqdm import tqdm  # プログラスバーは表示

# CoARiJのデータが保存されているディレクトリ
coarij_dir = 'data/CoARiJ'

# 対象年度(2014年から2018年)
years = list(range(2014, 2019))

# 辞書を読み込む
df = pd.read_csv('data/polarity_dic_News.csv', usecols=[1,2])
polarity_dic = dict(zip(df.word, df.score))

# MeCabのTaggerインスタンスは先に生成しておく
tagger = MeCab.Tagger("")

In [2]:
def get_postag(feature):
    return feature.split(',')[0]

def extract_words(node):
    words = []
    while node:
        postag = get_postag(node.feature)
        if postag.startswith('名詞'):
            words.append(node.surface)
        node = node.next
    return words

def parse_file(path, tagger):
    words_all = []
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            node = tagger.parseToNode(line)
            words = extract_words(node)
            words_all.extend(words)
    return words_all

In [3]:
# 年ごとに処理
year_hash = {}

for y in years:
    # ファイル一覧を得る
    paths = glob.glob(
        f'{coarij_dir}/{y}/docs/**/**_business_overview_of_result.txt',
        recursive=True)

    # ファイルごとに処理
    scores = []
    for path in tqdm(paths):
        words = parse_file(path, tagger)
        score = sum([polarity_dic.get(word, 0) for word in words])
        scores.append(score)

    # 用意した辞書型に保存
    year_hash[y] = scores

# 集計したデータをjsonlとして出力
with open('polarity_scores_mecab.jsonl', 'w') as f:
    for y in years:
        output = {'year': y,
                  'sum': sum(year_hash[y]),
                  'scores': year_hash[y]}
        f.write("{}\n".format(json.dumps(output)))

100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 83.39it/s]
